Vedi se è possibile correggere anche i riferimenti sintattici di Diaparser per i periodi spezzati (cambiare cioè il numero da vecchio a nuovo). Forse serve un mapping

In [1]:
import copy
import re
import xml.etree.ElementTree as ET
import stanza
from diaparser.parsers import Parser
parser = Parser.load('it_isdt.dbmdz-electra-xxl')

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [2]:
nlp = stanza.Pipeline(lang='it', processors ='tokenize')

2021-07-04 12:12:42 WARNING: Language it package default expects mwt, which has been added
2021-07-04 12:12:42 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2021-07-04 12:12:42 INFO: Use device: cpu
2021-07-04 12:12:42 INFO: Loading: tokenize
2021-07-04 12:12:42 INFO: Loading: mwt
2021-07-04 12:12:42 INFO: Done loading processors!


In [3]:
def scrittura_file(periodo, token_counter, indici_da_saltare, cue, scope):
    with open(percorso_file, 'a', encoding='utf-8') as file:
        file.write(periodo[0] + '\n' + periodo[1] + '\n')
        for riga in periodo[2:]:
            file.write(riga)
            # if riga[0] != '#': # aggiunto per i periodi più lunghi di 490 parole sintattiche, che contengono righe di commento anche in mezzo alla lista (non solo nelle prime due posizioni)
            if len(indici_da_saltare) > 0:
                numero_token_in_periodo = riga.split('\t')[0]
                da_annotare = True
                if '-' in numero_token_in_periodo:
                    file.write('\t_\t_\t_\n')
                    da_annotare = False
                else:
                    numero_token_in_periodo = int(numero_token_in_periodo)
                    for coppia_indici in indici_da_saltare:
                        if numero_token_in_periodo >= coppia_indici[0] and numero_token_in_periodo <= coppia_indici[1]:
                            file.write("\t{}\tDESCR.\tDESCR.\n".format(token_counter))
                            da_annotare = False
                            token_counter += 1
                        break
                if da_annotare:
                    file.write("\t{}\t{}\t{}\n".format(token_counter, cue, scope))
                    token_counter += 1
            else:
                if '-' in riga.split('\t')[0]:
                    file.write('\t_\t_\t_\n')
                else:
                    file.write("\t{}\t{}\t{}\n".format(token_counter, cue, scope))
                    token_counter += 1
            #else:
            #    file.write('\n')
    return token_counter

In [4]:
"""
'periodo' è una lista di stringhe. La lista è il periodo dell'intervento, ottenuto con diaparser e in formato CONLL-X; 
le stringhe contenute sono invece le righe del periodo stesso (ottenute usando come separatore del periodo '\n')
"""
def annotazione_periodo(token_counter, periodo):
    indici_da_saltare = []
    sequenza_descrittiva = {}
    for riga in [line for line in periodo if line[0] != '#']: # per prendere in considerazione i periodi spezzati, penso basti togliere le altre righe con # da questo ciclo,
                                                            # tanto il matching con il testo Connl-x viene fatto sulla base del numero del token, 
                                                            # che è salvato fra le chiavi (e non sulla base dell'indice dell'elemento o della chiave)
        sequenza_descrittiva[riga.split('\t')[0]] = riga.split('\t')[1]
    for match in re.finditer(r'(\(.*?\))', " ".join(sequenza_descrittiva.values())):
        print("È una sequenza descrittiva?[d/descr/no]")
        print(match[0])
        print()
        da_analizzare = input()
        input_errato = True
        while input_errato:
            if da_analizzare in ('d','D','descr.','descr','DESCR','DESCR.'):
                input_errato = False
                indice_chiave_iniziale = (" ".join(sequenza_descrittiva.values()))[:match.span()[0]+1].count(" ")
                indice_chiave_finale = (" ".join(sequenza_descrittiva.values()))[:match.span()[1]].count(" ")
                indici_da_saltare.append((int(list(sequenza_descrittiva.keys())[indice_chiave_iniziale]), int(list(sequenza_descrittiva.keys())[indice_chiave_finale])))
            elif da_analizzare in ('N','n','no','No','NO','nO'):
                input_errato = False
            else:
                print("Input errato. Riprova [d/descr/no]:")
                print(match)
                da_analizzare = input()
    righe_testo = [riga for riga in periodo if '# text =' in riga]
    for numero_riga_testo, riga in enumerate(righe_testo):
        print(riga[9:])
        da_analizzare = input('Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]')
        input_errato = True
        while input_errato is True:
            if da_analizzare not in ('N','n','no','No','NO','nO','','sì','si','Sì','s','S','SI','dubbio', 'dub', 'forse','?','descr.', 'DESCR.', 'd', 'descr', 'DESCR'):
                print("Input errato. Riprova.")
                print(riga[9:])
                da_analizzare = input("Vuoi analizzare questo periodo o blocco di periodo?[sì/no/forse]")
            else:
                indice_iniziale = periodo.index(riga)-1
                if numero_riga_testo < len(righe_testo)-1:
                    indice_finale = periodo.index(righe_testo[numero_riga_testo+1])-1
                else:
                    indice_finale = len(periodo)
                blocco_periodo = periodo[indice_iniziale:indice_finale]
                if da_analizzare in ('N','n','no','No','NO','nO',''):
                    input_errato = False
                    token_counter = scrittura_file(blocco_periodo, token_counter, indici_da_saltare,'O','_')
                    print("Hai risposto di no.")    
                elif da_analizzare in ('sì','si','Sì','s','S','SI'):
                    input_errato = False
                    token_counter = scrittura_file(blocco_periodo, token_counter, indici_da_saltare,' ',' ')
                    print("Hai risposto di sì.")
                elif da_analizzare in ('dubbio', 'dub', 'forse','?'):
                    input_errato = False
                    token_counter = scrittura_file(blocco_periodo, token_counter, indici_da_saltare,'?','?')
                    print("Hai risposto forse.")
                elif da_analizzare in ('descr.', 'DESCR.', 'd', 'descr', 'DESCR'):
                    input_errato = False
                    token_counter = scrittura_file(blocco_periodo, token_counter, indici_da_saltare, 'DESCR.','DESCR.')
                    print("Hai risposto 'descrizione'.")         
    return token_counter

In [5]:
def analisi_periodi(numero_periodo, token_counter, intervento):
    def analisi_periodo_lungo():
        global log ####
        periodo_lungo = []
        numero_iterazioni = int(len(lista_parole)/lunghezza_blocco)
        if len(lista_parole)%lunghezza_blocco != 0:
            numero_iterazioni += 1
        indice_parola_iniziale = 0
        log['numero blocchi previsto (periodo {})'.format(log['counter_periodi'])] = numero_iterazioni
        log['lunghezza lista parole (periodo {})'.format(log['counter_periodi'])] = len(lista_parole)
        for i in range(numero_iterazioni):
            indice_iniziale_blocco = int(lista_parole[indice_parola_iniziale].misc[lista_parole[indice_parola_iniziale].misc.find("=")+1:lista_parole[indice_parola_iniziale].misc.find("|")])
            if i < numero_iterazioni-1:
                # indice_finale_blocco dev'essere escluso: è l'indice del primo carattere dopo la parola
                indice_finale_blocco = int(lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc[lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc.rfind("=")+1:])
                blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_blocco]
                indice_parola_iniziale += lunghezza_blocco
            else:
                blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_periodo]
            log['blocco periodo no. {} (periodo {}):'.format(i, log['counter_periodi'])] = blocco_periodo ###
            periodo_diaparser = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(blocco_periodo, text='it').sentences]
            if len(periodo_diaparser) > 1:
                print("Diaparser ha risplittato un blocco di periodo. Procedo a riunirlo.")
                for periodo in periodo_diaparser[1:]:
                    periodo_diaparser[0].extend(periodo)
            periodo_diaparser = periodo_diaparser[0]
            periodo_diaparser.insert(0,'# sent_id = *.{}'.format(i+1))
            periodo_diaparser.insert(1, "# text = " + blocco_periodo)
            periodo_lungo.append(periodo_diaparser)
        return periodo_lungo
    global log ###
    log = {} ###
    log['counter_periodi'] = 0 ####
    testo_intervento = intervento.text
    diz_periodi = {}
    doc = nlp(testo_intervento)
    indice_finale_periodo_precedente = 0
    lunghezza_blocco = 0
    for num_periodo, periodo in enumerate(doc.sentences):
        log['counter_periodi'] += 1 ####
        lista_parole = periodo.words
        if len(lista_parole) >= 200:
            errore_input = True
            print(f'Trovato un periodo più lungo di 200 parole sintattiche: contiene un elenco? [sì/no]\n {periodo.text}')
            while errore_input == True:
                va_spezzata = input()
                if va_spezzata.strip() in ('Sì','sì','si','Si','s','no','No','n'):
                    errore_input = False
                    if 's' in va_spezzata.lower():
                        print('Hai risposto di sì.')
                        lunghezza_blocco = 200
                    else:
                        print('Hai risposto di no.')
                else:
                    print('Input errato. Riprova:')
        if lunghezza_blocco == 200:
            with open('periodi_spezzati.txt','a') as file:
                file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\n")
            print("Un periodo è stato spezzato perché troppo lungo. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzati.txt'")
            indice_iniziale_periodo = int(lista_parole[0].misc[lista_parole[0].misc.find("=")+1:lista_parole[0].misc.find("|")])
            indice_finale_periodo = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            if indice_iniziale_periodo != indice_finale_periodo_precedente:
                if re.fullmatch(r'\s+',testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]) is None:
                    diz_periodi[indice_finale_periodo_precedente] = testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]
            indice_finale_periodo_precedente = indice_finale_periodo # cambiato da = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            errore_diaparser = True
            while errore_diaparser == True:
                errore_diaparser = False
                try:
                    diz_periodi[indice_iniziale_periodo] = analisi_periodo_lungo()
                    log['diz_periodi'] = diz_periodi ####
                except IndexError:
                    errore_diaparser = True
                    lunghezza_blocco -= 20
        elif len(lista_parole) > 480:
            indice_iniziale_periodo = int(lista_parole[0].misc[lista_parole[0].misc.find("=")+1:lista_parole[0].misc.find("|")])
            indice_finale_periodo = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            if indice_iniziale_periodo != indice_finale_periodo_precedente:
                if re.fullmatch(r'\s+',testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]) is None:
                    diz_periodi[indice_finale_periodo_precedente] = testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]
            indice_finale_periodo_precedente = indice_finale_periodo # cambiato da = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            try:
                lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(testo_intervento[indice_iniziale_periodo:indice_finale_periodo], text='it').sentences]
                for counter, periodo_diaparser in enumerate(lista_periodi):
                    diz_periodi[indice_iniziale_periodo + counter] = periodo_diaparser
                if len(lista_periodi) > 1:
                    with open('periodi_spezzati.txt','a') as file:
                        file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\tN.B.: In questo caso, non è stato spezzato un blocco del periodo, ma il periodo individuato da Stanza, \n\tche aveva più di 480 word, è stato scisso in più periodi da diaparser. L'analisi sintattica dovrebbe comunque essere buona.\n\n")
                    print("Un periodo di Stanza > 480 è stato diviso in più periodi da diaparser. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzati.txt'")
            except IndexError:
                with open('periodi_spezzati.txt','a') as file:
                    file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\n")
                print("Un periodo è stato spezzato perché troppo lungo. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzati.txt'")
                lunghezza_blocco = 480
                errore_diaparser = True
                while errore_diaparser == True:
                    errore_diaparser = False
                    try:
                        diz_periodi[indice_iniziale_periodo] = analisi_periodo_lungo()
                        log['diz_periodi'] = diz_periodi ####
                    except IndexError:
                        errore_diaparser = True
                        lunghezza_blocco -= 20
        # in caso l'intervento si chiuda con uno o più periodi che non vengono spezzati
        if num_periodo == len(doc.sentences)-1:
            indice_finale_periodo = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            if indice_finale_periodo != indice_finale_periodo_precedente:
                diz_periodi[indice_finale_periodo_precedente] = testo_intervento[indice_finale_periodo_precedente:]
    if bool(diz_periodi) is True:
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        for indice in indici_periodi_analizzati:
            if isinstance (diz_periodi[indice], str):
                blocco_testo_intervento = diz_periodi[indice]
                for counter, periodo_diaparser in enumerate([str(sentence).split('\n')[:-1] for sentence in parser.predict(blocco_testo_intervento, text='it').sentences]):
                    diz_periodi[indice + counter] = periodo_diaparser
        log['diz_periodi'] = diz_periodi ####
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        for indice in indici_periodi_analizzati:
            if isinstance (diz_periodi[indice][0], list):
                periodo_completo = []
                for num_blocco, blocco_periodo in enumerate(diz_periodi[indice]):
                    blocco_periodo[0] = blocco_periodo[0].replace('*',str(numero_periodo))
                    if num_blocco == 0:
                        numero_token_precedente = blocco_periodo[-1].split('\t')[0]
                        # se l'ultimo token del primo blocco è un mwt, basta risalire al numero del token precedente, 
                        # che sarà sicuramente un token semplice: non si danno mai due mwt di fila
                        if '-' in numero_token_precedente:
                            numero_token_precedente = int(blocco_periodo[-2].split('\t')[0])
                        else:
                            numero_token_precedente = int(numero_token_precedente)
                    else:
                        num_riga = 2
                        for riga in blocco_periodo[2:]:
                            numero_vecchio = riga.split('\t')[0]
                            if '-' in numero_vecchio:
                                primo_numero = numero_vecchio[:numero_vecchio.find('-')]
                                secondo_numero = numero_vecchio[numero_vecchio.find('-')+1:]
                                lunghezza_mwt = int(secondo_numero) - int(primo_numero)
                                nuovo_numero = str(numero_token_precedente+1) + '-' + str(numero_token_precedente+1+lunghezza_mwt)
                            else:
                                nuovo_numero = str(numero_token_precedente+1)
                                numero_token_precedente += 1
                            blocco_periodo[num_riga] = riga.replace(numero_vecchio, nuovo_numero, 1)
                            num_riga += 1
                    periodo_completo.extend(blocco_periodo)
                token_counter = annotazione_periodo(token_counter,periodo_completo)
            else:
                numero_periodo_annotato = re.search(r'[0-9]+', diz_periodi[indice][0])[0]
                diz_periodi[indice][0].replace(numero_periodo_annotato, str(numero_periodo))
                token_counter = annotazione_periodo(token_counter, diz_periodi[indice])
            numero_periodo += 1
    else:
        lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(testo_intervento, text='it').sentences]
        for periodo in lista_periodi:
            token_counter = annotazione_periodo(token_counter, periodo)

In [6]:
def gestione_intervento(intervento):
    with open(percorso_file,'a', encoding='utf-8') as file:
        riga_iniziale = '# INTERVENTO NUMERO {}\n'.format(numero_intervento)
        file.write(riga_iniziale)
        file.write('# id="' + intervento.attrib['id'] + '"' + ' cognomeNome="' + intervento.attrib['cognomeNome'] + '"')
        if intervento.attrib['tipo'] == 'parlamentare':
            file.write(' partito="' + intervento.attrib['partito'] + '"\n')
        else:
            if 'nominativo' in intervento.attrib:
                file.write(' nominativo="' + intervento.attrib['nominativo'] + '"')
            file.write("\n")
    analisi_periodi(1, 1, intervento)

In [7]:
numero_legislatura = str(18)
numero_seduta = str(6)
tree = ET.parse('Resoconti/legislatura_{}/seduta_{}.xml'.format(numero_legislatura, numero_seduta))
seduta = tree.getroot()
seduta_da_analizzare = False
if len(list(seduta.iter("intervento"))) > 0:
    seduta_da_analizzare = True

In [8]:
if seduta_da_analizzare:
    percorso_file = 'Resoconti_annotati/legislatura_{}/preannotati/{}.tsv'.format(numero_legislatura, numero_seduta)
    file_esiste = False
    try:
        with open(percorso_file, 'x', encoding='utf-8') as file:
            legenda = "ID\tFORM\tLEMMA\tCPOS\tPOS\tFEATS\tHEAD\tDEPREL\tPHEAD\tPDEPREL\tID_TOKEN_NELL_INTERVENTO\tCUE\tSCOPE\n"
            file.write(legenda)
    except FileExistsError:
        print(f"Attenzione! Il file relativo a questa seduta ({numero_seduta}) esiste già. Si tenterà di riprendere da dove si è lasciato, ma occorre verificare che il periodo proposto come inizio sia quello che effettivamente segue all'ultimo annotato.\n")
        file_esiste = True
        finisce_con_titolo = False
        with open(percorso_file, 'r', encoding='utf-8') as file:
            riga = file.readline()
            ultima_riga = ""
            while(riga != ""):
                if '# INTERVENTO NUMERO' in riga:
                    num_int_finale = int(re.search(r'[0-9]+', riga)[0])
                    num_periodo_finale = 0
                    intervento = list(seduta.iter("intervento"))[num_int_finale]
                elif '# text =' in riga:
                    testo_da_cancellare = riga[riga.index('=')+2:].strip('\n\t ').strip('.')
                    # purtroppo, non è detto che il testo finale sia quello da cui ripartire, perché può accadere che 
                    # nella scrittura nel file qualche parola vada persa --> bisogna comunque controllare
                    if testo_da_cancellare in intervento.text:
                        # str.index(substr) restituisce l'indice del primo carattere della prima occorrenza della sottostringa 
                        # --> no problemi di (improbabili) duplicati nel testo
                        intervento.text = intervento.text[intervento.text.index(testo_da_cancellare)+len(testo_da_cancellare):]
                elif '# sent_id' in riga:
                    num_periodo_finale = int(re.search(r'[0-9]+', riga)[0])
                ultima_riga = riga
                riga = file.readline()
        if '# Titolo' in ultima_riga:
            finisce_con_titolo = True
        elif '# ' in ultima_riga:
            token_counter = 1
        else:
            token_counter = int(ultima_riga.split("\t")[10])+1
        if finisce_con_titolo is False:
            numero_intervento = num_int_finale
            intervento.text = intervento.text.lstrip('.').strip(' \n\t')
            analisi_periodi(num_periodo_finale+1, token_counter, intervento) # eliminato "token_counter =" perché non mi pare lo riusassi dopo
    if file_esiste:
        numero_intervento = num_int_finale+1
        if finisce_con_titolo:
            el_iniziale = list(seduta.iter('intervento'))[num_int_finale+1]
            for counter, Element in enumerate(seduta.find('resoconto').iter()):
                if Element.tag == 'intervento' and Element.attrib['id'] == el_iniziale.attrib['id']:
                    num_el_iniziale = counter
                    break
        else:
            el_iniziale = list(seduta.iter('intervento'))[num_int_finale]
            for counter, Element in enumerate(seduta.find('resoconto').iter()):
                if Element.tag == 'intervento' and Element.attrib['id'] == el_iniziale.attrib['id']:
                    num_el_iniziale = counter + 1
                    break
        for Element in list(seduta.find('resoconto').iter())[num_el_iniziale:]:
            if Element.tag == 'intervento':
                gestione_intervento(Element)
                numero_intervento += 1
            elif Element.tag == 'dibattito':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file, 'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
            elif Element.tag == 'fase':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
    else:
        with open(percorso_file, 'a', encoding='utf-8') as file:
            file.write("#")
            for key, value in seduta.attrib.items():
                file.write(' ' + key + '="' + value +'"')
            file.write("\n")
        numero_intervento = 0
        for Element in list(seduta.find('resoconto').iter()):
            if Element.tag == 'intervento':
                gestione_intervento(Element)
                numero_intervento += 1
            elif Element.tag == 'dibattito':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
            elif Element.tag == 'fase':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")

Attenzione! Il file relativo a questa seduta (6) esiste già. Si tenterà di riprendere da dove si è lasciato, ma occorre verificare che la periodo proposta come inizio sia quella che effettivamente segue all'ultima annotata
Signor Presidente, a più di tre giorni dall'attacco aereo che ha colpito la Siria, l'opinione pubblica continua a porsi alcuni interrogativi fondamentali:
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
l'attacco era veramente necessario e aveva la legittimità ai sensi del diritto internazionale?
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Qual è il ruolo che compete al nostro Paese in questa situazione?
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Sono interrogativi fondamentali perché l'Italia, per la sua collocazione geografica, per i suoi interessi strategici, per la sua scelta di campo occidentale, che da settant'anni è prima di tutto una scel

Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]?
Hai risposto forse.
Per chi, invece, come noi, crede nel diritto internazionale, questa deve essere un'occasione nella quale le Nazioni Unite debbono tornare ad essere il luogo di garanzia e legalità internazionale.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
In questo quadro, da occidentali alleati dell'America e partner dell'Europa, siamo fermamente convinti del fatto che la Russia di Putin vada vista come un partner strategico e non come avversario.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
Questo non significa che non vi possono essere visioni diverse o interessi in contrasto;
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
significa che tali contrasti vanno risolti dialogando e non in conflitto con Mosca, non rischiando incidenti sul terreno o imprevedibili escalation.
Vuoi analizzare questo

Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
Ora, questo non significa, ovviamente, mettere in discussione le proprie alleanze;
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
significa, però, avere la pretesa di partecipare alle organizzazioni internazionali con dignità.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
L'Italia fa parte della NATO, dell'Alleanza atlantica ed è giusto che mantenga i suoi impegni internazionali;
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
però, la partecipazione all'Alleanza atlantica prevede due cose: l'impegno a una difesa collettiva, ossia l'intervento automatico in caso di aggressione di uno degli Stati membri, e la possibilità di concordare azioni e interventi comuni per specifiche esigenze di sicurezza comune o su mandato delle Nazioni Unite.
Vuoi analizzare questo periodo o blocco di perio

Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
sono i nostri interessi geopolitici?
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Permettetemi di avere qualche dubbio.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Allora, penso che l'Italia, oggi, debba scegliere se difendere il diritto internazionale e, quindi, dire “no” alle azioni militari unilaterali, oppure stabilire che vige la legge del più forte, dove il diritto internazionale lo stabilisce chi ha la maggiore capacità militare.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
Capisco che possa essere utile a potenze nucleari come gli Stati Uniti, la Francia o la Gran Bretagna.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Non mi è esattamente chiaro perché dovrebbe essere utile, a una nazione militarmente un tantino - diciamo - meno 

Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Come mezzo di risoluzione delle controversie internazionali:
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
questa deve essere la guida, la linea guida per dare il nostro contributo, il contributo dell'Italia, di un grande Paese, alla soluzione della crisi siriana, impedendo un allargamento del conflitto all'intera area del Medioriente, con conseguenze disastrose sotto ogni punto di vista, a cominciare da quello delle popolazioni civili.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
È una sequenza descrittiva?[d/descr/no]
( Applausi dei di i deputati del di il gruppo Liberi e Uguali )

d
Tacciano, quindi, le armi e lavorino con determinazione e volontà positive le diplomazie dei maggiori Paesi, con l'Italia da protagonista in prima fila (Applausi dei deputati del gruppo Liberi e Uguali).
Vuoi analizzare questo periodo o blo

Hai risposto di sì.
È questa la domanda che tutto il mondo si deve porre: come contribuire alla fine di questa guerra, ma a creare una pace che sia duratura in tutto il mondo.
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Questo è il ruolo che l'Italia deve avere, questo è il ruolo che l'Europa deve avere nell'adesione convinta al Patto Atlantico;
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
È una sequenza descrittiva?[d/descr/no]
( Applausi dei di i deputati del di il gruppo Misto - Noi con l' Italia e di deputati del di il gruppo Forza Italia - Berlusconi Presidente )

d
questo è il contributo che noi possiamo dare, senza falsi infingimenti e smettendola con le battaglie ideologiche magari di casa nostra, che non servono a nessuno (Applausi dei deputati del gruppo Misto-Noi con l'Italia e di deputati del gruppo Forza Italia-Berlusconi Presidente).
Vuoi analizzare questo periodo o blocco di periodo? [sì/n

Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Devo dire che ieri mattina ho tenuto un presidio…
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
Io sono segretario nazionale del Partito Pensionati eletto in Forza Italia, con grande riconoscenza per questo movimento e per il suo Presidente Silvio Berlusconi;
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]
Hai risposto di no.
e mi sono recato davanti all'INPS di Milano, via Gustavo Silva 38, per manifestare, insieme a tanti attivisti pensionati, con bandiere nostre e di Forza Italia, bandiere del Partito Pensionati e di Forza Italia, perché venga consegnato a tutti i pensionati ogni anno il prospetto della propria pensione, che da quattro anni a questa parte non viene più loro consegnato:
Vuoi analizzare questo periodo o blocco di periodo? [sì/no/forse]s
Hai risposto di sì.
talché si presentano davanti all'ufficio postale o alla banca trepidant

Statistiche lunghezze periodi:

In [23]:
# codice usato per generare la lista delle lunghezze delle periodi delle prime 29 sedute
'''
lunghezze_periodi = []
for numero_seduta in range(1,29+1):
    numero_legislatura = str(18)
    tree = ET.parse('Resoconti/legislatura_{}/seduta_{}.xml'.format(numero_legislatura, numero_seduta))
    seduta = tree.getroot()
    seduta_da_analizzare = False
    if len(list(seduta.iter("intervento"))) > 0:
        for Element in list(seduta.find('resoconto').iter()):
            if Element.tag == 'intervento':
                doc = nlp(Element.text)
                for periodo in doc.sentences:
                    if len(periodo.words) > 200:
                        lunghezze_periodi.append((len(periodo.words),periodo.text))
                    else:
                        lunghezze_periodi.append(len(periodo.words))
'''

In [3]:
import json
with open('lunghezze_periodi_resoconti.txt', 'r', encoding='utf-8') as file:
    lunghezze_periodi = json.load(file)

In [17]:
media_lunghezze = 0
for item in lunghezze_periodi:
    if isinstance(item, list):
        media_lunghezze += item[0]
    else:
        media_lunghezze += item
media_lunghezze = media_lunghezze/len(lunghezze_periodi)
print(f"Lunghezza media delle periodi: {media_lunghezze}")

Lunghezza media delle periodi: 36.81321825818064


In [18]:
numero_lunghezze_maggiori_200 = len([i for i in lunghezze_periodi if isinstance(i, list)])
print(f'Numero periodi più lunghe di 200 parole sintattiche: {numero_lunghezze_maggiori_200}')

Numero periodi più lunghe di 200 parole sintattiche: 44


In [28]:
tasso_lungh_maggiori_200 = (numero_lunghezze_maggiori_200/len(lunghezze_periodi))*100
print(f'Tasso periodi più lunghe di 200: {str(tasso_lungh_maggiori_200)[:5]}%')

Tasso periodi più lunghe di 200: 0.2%


In [34]:
solo_lunghezze = []
for item in lunghezze_periodi:
    if isinstance(item, list):
        solo_lunghezze.append(item[0])
    else:
        solo_lunghezze.append(item)
numero_lunghezze_maggiori_480 = len([l for l in solo_lunghezze if l > 480])
print(f'Numero periodi più lunghe di 480 parole sintattiche: {numero_lunghezze_maggiori_480}')

Numero periodi più lunghe di 480 parole sintattiche: 19


In [36]:
tasso_lungh_maggiori_480 = (numero_lunghezze_maggiori_480/len(lunghezze_periodi))*100
print(f'Tasso periodi più lunghe di 480: {str(tasso_lungh_maggiori_480)[:6]}%')

Tasso periodi più lunghe di 480: 0.08%


Provo a vedere se, diminuendo la lunghezza del testo, diaparser è in grado di gestirlo (ho dovuto assegnare il testo a una nuova variabile perché dopo l'esecuzione delle celle precedenti ho chiuso la sessione - ho recuperato il testo dall'output delle celle):

In [12]:
testo_incriminato = "L'Abbate Giuseppe La Marca Francesca Labriola Vincenza Lacarra Marco Lapia Mara Latini Giorgia Lattanzio Paolo Lazzarini Arianna Legnaioli Donatella Lepri Stefano Librandi Gianfranco Licatini Caterina Liuni Marzio Liuzzi Mirella Lo Monte Carmelo Locatelli Alessandra Lolini Mario Lollobrigida Francesco Lombardo Antonio Longo Fausto Lorefice Marialucia Lorenzin Beatrice Lorenzoni Eva Lorenzoni Gabriele Losacco Alberto Lotti Luca Lovecchio Giorgio Lucaselli Ylenja Lucchini Elena Lupi Maurizio Maccanti Elena Macina Anna Madia Maria Anna Maggioni Marco Magi Riccardo Maglione Pasquale Mammì Stefania Manca Alberto Manca Gavino Mancini Claudio Mandelli Andrea Maniero Alvise Manzato Franco Manzo Teresa Maraia Generoso Marattin Luigi Marchetti Riccardo Augusto Mariani Felice Marin Marco Marino Bernardo Marrocco Patrizia Martina Maurizio Martinciglio Vita Martino Antonio Marzana Maria Maschio Ciro Masi Angela Maturi Filippo Mauri Matteo Mazzetti Erica Melicchio Alessandro Melilli Fabio Meloni Giorgia Menga Rosa Miceli Carmelo Micillo Salvatore Migliore Gennaro Migliorino Luca Milanato Lorena Minardo Antonino Minniti Marco Misiti Carmelo Massimo Molinari Riccardo Mollicone Federico Molteni Nicola Montaruli Augusta Mor Mattia Morani Alessia Morassut Roberto Morelli Alessandro Morgoni Mario Morrone Jacopo Moschioni Daniele Mugnai Stefano Mulè Giorgio Mura Andrea Mura Romina Murelli Elena Muroni Rossella Musella Graziano Nanni Iolanda Napoli Osvaldo Nappi Silvana Nardi Martina Navarra Pietro Nesci Dalila Nevi Raffaele Nitti Michele Nobili Luciano Noja Lisa Novelli Roberto Occhionero Giuseppina Occhiuto Roberto Olgiati Riccardo Orfini Matteo Orlando Andrea Orrico Anna Laura Orsini Andrea Osnato Marco Padoan Pietro Carlo Pagani Alberto Pagano Alessandro Pagano Ubaldo Paita Raffaella Palazzotto Erasmo Pallini Maria Palmieri Antonio Palmisano Valentina Panizzut Massimiliano Paolini Luca Rodolfo Papiro Antonella Parentela Paolo Parisse Martina Parolo Ugo Pastorino Luca Patassini Tullio Patelli Cristina Paternoster Paolo Paxia Maria Laura Pedrazzini Claudio Pella Roberto Pellicani Nicola Pentangelo Antonio Perantoni Mario Perconti Filippo Giuseppe Perego Di Cremnago Matteo Pettarin Guido Germano Pettazzi Lino Pezzopane Stefania Piastra Carlo Picchi Guglielmo Piccoli Nardelli Flavia Pignatone Dedalo Cosimo Gaetano Pini Giuditta Pittalis Pietro Pizzetti Luciano Plangger Albrecht Polidori Catia Pollastrini Barbara Polverini Renata Porchietto Claudia Portas Giacomo Potenti Manfredi Prestigiacomo Stefania Prestipino Patrizia Pretto Erik Umberto Prisco Emanuele Provenza Nicola Quartapelle Procopio Lia Racchella Germano Raciti Fausto Raduzzi Raphael Raffa Angela Raffaelli Elena Rampelli Fabio Ribolla Alberto Ricciardi Riccardo Ripani Elisabetta Rixi Edoardo Rizzetto Walter Rizzo Gianluca Rizzo Nervo Luca Rizzone Marco Romaniello Cristian Romano Andrea Romano Paolo Nicolò Rosato Ettore Rospi Gianluca Rossello Cristina Rossi Andrea Rossini Emanuela Rossini Roberto Rosso Roberto Rostan Michela Rotelli Mauro Rotondi Gianfranco Rotta Alessia Ruffino Daniela Ruggieri Andrea Ruggiero Francesca Anna Ruocco Carla Russo Giovanni Russo Paolo Saccani Jotti Gloria M.R. Saitta Eugenio Salafia Angela Saltamartini Barbara Sangregorio Eugenio Santelli Jole Sapia Francesco Sarli Doriana Sarro Carlo Sarti Giulia Sasso Rossano Savino Elvira Savino Sandra Scagliusi Emanuele Scalfarotto Ivan Scanu Lucia Scerra Filippo Schirò Angela Schullian Manfred Scoma"

In [10]:
ds = parser.predict(" ".join(testo_incriminato.split(' ')[:400]), text = 'it')

IndexError: The shape of the mask [1, 710, 710] at index 1 does not match the shape of the indexed tensor [1, 512, 512] at index 1

Anche con 401 parole (L'Abbate viene scomposto in L' + Abbate), diaparser dà errore.
Proviamo con 350:

In [13]:
ds = parser.predict(" ".join(testo_incriminato.split(' ')[:350]), text = 'it')

IndexError: The shape of the mask [1, 622, 622] at index 1 does not match the shape of the indexed tensor [1, 512, 512] at index 1

Con 201:

In [14]:
ds = parser.predict(" ".join(testo_incriminato.split(' ')[:200]), text = 'it')

In [15]:
ds

Dataset(n_sentences=1, n_batches=1, n_buckets=1)

In [20]:
ds.sentences

[# sent_id = 1
 # text = L'Abbate Giuseppe La Marca Francesca Labriola Vincenza Lacarra Marco Lapia Mara Latini Giorgia Lattanzio Paolo Lazzarini Arianna Legnaioli Donatella Lepri Stefano Librandi Gianfranco Licatini Caterina Liuni Marzio Liuzzi Mirella Lo Monte Carmelo Locatelli Alessandra Lolini Mario Lollobrigida Francesco Lombardo Antonio Longo Fausto Lorefice Marialucia Lorenzin Beatrice Lorenzoni Eva Lorenzoni Gabriele Losacco Alberto Lotti Luca Lovecchio Giorgio Lucaselli Ylenja Lucchini Elena Lupi Maurizio Maccanti Elena Macina Anna Madia Maria Anna Maggioni Marco Magi Riccardo Maglione Pasquale Mammì Stefania Manca Alberto Manca Gavino Mancini Claudio Mandelli Andrea Maniero Alvise Manzato Franco Manzo Teresa Maraia Generoso Marattin Luigi Marchetti Riccardo Augusto Mariani Felice Marin Marco Marino Bernardo Marrocco Patrizia Martina Maurizio Martinciglio Vita Martino Antonio Marzana Maria Maschio Ciro Masi Angela Maturi Filippo Mauri Matteo Mazzetti Erica Melicchio Alessandro

In [16]:
ds2 = parser.predict(" ".join(testo_incriminato.split(' ')[200:400]), text = 'it')

In [17]:
ds2

Dataset(n_sentences=1, n_batches=1, n_buckets=1)

In [21]:
ds2.sentences

[# sent_id = 1
 # text = Michele Nobili Luciano Noja Lisa Novelli Roberto Occhionero Giuseppina Occhiuto Roberto Olgiati Riccardo Orfini Matteo Orlando Andrea Orrico Anna Laura Orsini Andrea Osnato Marco Padoan Pietro Carlo Pagani Alberto Pagano Alessandro Pagano Ubaldo Paita Raffaella Palazzotto Erasmo Pallini Maria Palmieri Antonio Palmisano Valentina Panizzut Massimiliano Paolini Luca Rodolfo Papiro Antonella Parentela Paolo Parisse Martina Parolo Ugo Pastorino Luca Patassini Tullio Patelli Cristina Paternoster Paolo Paxia Maria Laura Pedrazzini Claudio Pella Roberto Pellicani Nicola Pentangelo Antonio Perantoni Mario Perconti Filippo Giuseppe Perego Di Cremnago Matteo Pettarin Guido Germano Pettazzi Lino Pezzopane Stefania Piastra Carlo Picchi Guglielmo Piccoli Nardelli Flavia Pignatone Dedalo Cosimo Gaetano Pini Giuditta Pittalis Pietro Pizzetti Luciano Plangger Albrecht Polidori Catia Pollastrini Barbara Polverini Renata Porchietto Claudia Portas Giacomo Potenti Manfredi Prestigi

In [18]:
ds3 = parser.predict(" ".join(testo_incriminato.split(' ')[400:]), text = 'it')

In [19]:
ds3

Dataset(n_sentences=1, n_batches=1, n_buckets=1)

In [22]:
ds3.sentences

[# sent_id = 1
 # text = Anna Ruocco Carla Russo Giovanni Russo Paolo Saccani Jotti Gloria M.R. Saitta Eugenio Salafia Angela Saltamartini Barbara Sangregorio Eugenio Santelli Jole Sapia Francesco Sarli Doriana Sarro Carlo Sarti Giulia Sasso Rossano Savino Elvira Savino Sandra Scagliusi Emanuele Scalfarotto Ivan Scanu Lucia Scerra Filippo Schirò Angela Schullian Manfred Scoma
 1	Anna	_	_	_	_	0	root	_	_
 2	Ruocco	_	_	_	_	1	flat:name	_	_
 3	Carla	_	_	_	_	1	conj	_	_
 4	Russo	_	_	_	_	3	flat:name	_	_
 5	Giovanni	_	_	_	_	1	conj	_	_
 6	Russo	_	_	_	_	7	flat:name	_	_
 7	Paolo	_	_	_	_	1	conj	_	_
 8	Saccani	_	_	_	_	7	flat:name	_	_
 9	Jotti	_	_	_	_	7	flat:name	_	_
 10	Gloria	_	_	_	_	7	conj	_	_
 11	M.R.	_	_	_	_	10	flat:name	_	_
 12	Saitta	_	_	_	_	10	flat:name	_	_
 13	Eugenio	_	_	_	_	10	conj	_	_
 14	Salafia	_	_	_	_	13	flat:name	_	_
 15	Angela	_	_	_	_	13	conj	_	_
 16	Saltamartini	_	_	_	_	17	flat:name	_	_
 17	Barbara	_	_	_	_	1	conj	_	_
 18	Sangregorio	_	_	_	_	17	flat:name	_	_
 19	Eugenio	_	_	_	_	17	co

<hr>

<h1> DA IGNORARE </h1>

In [6]:
# backup, comunque non avevi considerato il caso in cui l'intervento si chiude con una o più periodi che non sono
# troppo lunghe e quindi non vengono analizzate subito
'''
def analisi_periodi(numero_periodo, token_counter, intervento):
    global log ###
    log = {} ###
    log['counter_periodi'] = 0 ####
    testo_intervento = intervento.text
    diz_periodi = {}
    doc = nlp(testo_intervento)
    indice_finale_periodo_precedente = 0
    lunghezza_blocco = 0
    for periodo in doc.sentences:
        log['counter_periodi'] += 1 ####
        lista_parole = periodo.words
        if len(lista_parole) >= 200:
            errore_input = True
            print(f'Trovata una periodo più lunga di 200 parole sintattiche: contiene un elenco? [sì/no]\n {periodo.text}')
            while errore_input == True:
                va_spezzata = input()
                if va_spezzata.strip() in ('Sì','sì','si','Si','s','no','No','n'):
                    errore_input = False
                    if 's' in va_spezzata.lower():
                        print('Hai risposto di sì.')
                        lunghezza_blocco = 200
                    else:
                        print('Hai risposto di no.')
                else:
                    print('Input errato. Riprova:')
        if lunghezza_blocco == 200:
            with open('periodi_spezzate.txt','a') as file:
                file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\n")
            print("Una periodo è stata spezzata perché troppo lunga. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzate.txt'")
            indice_iniziale_periodo = int(lista_parole[0].misc[lista_parole[0].misc.find("=")+1:lista_parole[0].misc.find("|")])
            indice_finale_periodo = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            if indice_iniziale_periodo != indice_finale_periodo_precedente:
                if re.fullmatch(r'\s+',testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]) is None:
                    diz_periodi[indice_finale_periodo_precedente] = testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]
            indice_finale_periodo_precedente = indice_finale_periodo # cambiato da = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            periodo_lungo = []
            numero_iterazioni = int(len(lista_parole)/lunghezza_blocco)
            if len(lista_parole)%lunghezza_blocco != 0:
                numero_iterazioni += 1
            indice_parola_iniziale = 0
            log['numero blocchi previsto (periodo {})'.format(log['counter_periodi'])] = numero_iterazioni
            log['lunghezza lista parole (periodo {})'.format(log['counter_periodi'])] = len(lista_parole)
            for i in range(numero_iterazioni):
                indice_iniziale_blocco = int(lista_parole[indice_parola_iniziale].misc[lista_parole[indice_parola_iniziale].misc.find("=")+1:lista_parole[indice_parola_iniziale].misc.find("|")])
                if i < numero_iterazioni-1:
                    # indice_finale_blocco dev'essere escluso: è l'indice del primo carattere dopo la parola
                    indice_finale_blocco = int(lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc[lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc.rfind("=")+1:])
                    blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_blocco]
                    indice_parola_iniziale += lunghezza_blocco
                else:
                    blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_periodo]
                log['blocco periodo no. {} (periodo {}):'.format(i, log['counter_periodi'])] = blocco_periodo ###
                periodo_diaparser = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(blocco_periodo, text='it').sentences]
                if len(periodo_diaparser) > 1:
                    print("Diaparser ha risplittato un blocco di periodo. Procedo a riunirlo.")
                    for periodo in periodo_diaparser[1:]:
                        periodo_diaparser[0].extend(periodo)
                periodo_diaparser = periodo_diaparser[0]
                periodo_diaparser.insert(0,'# sent_id = *.{}'.format(i+1))
                periodo_diaparser.insert(1, "# text = " + blocco_periodo)
                periodo_lungo.append(periodo_diaparser)
            diz_periodi[indice_iniziale_periodo] = periodo_lungo
            log['diz_periodi'] = diz_periodi ####        
        elif len(lista_parole >= 480):
            indice_iniziale_periodo = int(lista_parole[0].misc[lista_parole[0].misc.find("=")+1:lista_parole[0].misc.find("|")])
            indice_finale_periodo = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            if indice_iniziale_periodo != indice_finale_periodo_precedente:
                if re.fullmatch(r'\s+',testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]) is None:
                    diz_periodi[indice_finale_periodo_precedente] = testo_intervento[indice_finale_periodo_precedente:indice_iniziale_periodo]
            indice_finale_periodo_precedente = indice_finale_periodo # cambiato da = int(lista_parole[-1].misc[lista_parole[-1].misc.rfind("=")+1:])
            try:
                lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(testo_intervento[indice_iniziale_periodo:indice_finale_periodo], text='it').sentences]
                for counter, periodo_diaparser in enumerate(lista_periodi):
                    diz_periodi[indice_iniziale_periodo + counter] = periodo_diaparser
                if len(lista_periodi) > 1:
                    with open('periodi_spezzate.txt','a') as file:
                        file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\tN.B.: In questo caso, non è stato spezzato un blocco della periodo, ma la periodo individuata da Stanza, \n\tche aveva più di 480 word, è stata scissa in più periodi da diaparser. L'analisi sintattica dovrebbe comunque essere buona.\n\n")
                    print("Una periodo di Stanza > 480 è stata divisa in più periodi da diaparser. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzate.txt'")
            except IndexError:
                with open('periodi_spezzate.txt','a') as file:
                    file.write(f"No. seduta: {seduta.attrib['numero']}, \nID intervento: {intervento.attrib['id']},\n\tInizio periodo: {periodo.text[:15]}\n\tFine periodo: {periodo.text[-15:]}\n\n")
                print("Una periodo è stata spezzata perché troppo lunga. I suoi dati identificativi sono stati aggiunti al file 'periodi_spezzate.txt'")
                errore_diaparser = True
                lunghezza_blocco = 479
                while errore_diaparser == True:
                    errore_diaparser = False
                    try:
                        periodo_lungo = []
                        numero_iterazioni = int(len(lista_parole)/lunghezza_blocco)
                        if len(lista_parole)%lunghezza_blocco != 0:
                            numero_iterazioni += 1
                        indice_parola_iniziale = 0
                        log['numero blocchi previsto (periodo {})'.format(log['counter_periodi'])] = numero_iterazioni
                        log['lunghezza lista parole (periodo {})'.format(log['counter_periodi'])] = len(lista_parole)
                        for i in range(numero_iterazioni):
                            indice_iniziale_blocco = int(lista_parole[indice_parola_iniziale].misc[lista_parole[indice_parola_iniziale].misc.find("=")+1:lista_parole[indice_parola_iniziale].misc.find("|")])
                            if i < numero_iterazioni-1:
                                # indice_finale_blocco dev'essere escluso: è l'indice del primo carattere dopo la parola
                                indice_finale_blocco = int(lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc[lista_parole[indice_parola_iniziale+(lunghezza_blocco - 1)].misc.rfind("=")+1:])
                                blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_blocco]
                                indice_parola_iniziale += lunghezza_blocco
                            else:
                                blocco_periodo = testo_intervento[indice_iniziale_blocco:indice_finale_periodo]
                            log['blocco periodo no. {} (periodo {}):'.format(i, log['counter_periodi'])] = blocco_periodo ###
                            periodo_diaparser = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(blocco_periodo, text='it').sentences]
                            if len(periodo_diaparser) > 1:
                                print("Diaparser ha risplittato un blocco di periodo. Procedo a riunirlo.")
                                for periodo in periodo_diaparser[1:]:
                                    periodo_diaparser[0].extend(periodo)
                            periodo_diaparser = periodo_diaparser[0]
                            periodo_diaparser.insert(0,'# sent_id = *.{}'.format(i+1))
                            periodo_diaparser.insert(1, "# text = " + blocco_periodo)
                            periodo_lungo.append(periodo_diaparser)
                        diz_periodi[indice_iniziale_periodo] = periodo_lungo
                        log['diz_periodi'] = diz_periodi ####
                    except IndexError:
                        errore_diaparser = True
                        lunghezza_blocco -= 20
    if bool(diz_periodi) is True:
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        for indice in indici_periodi_analizzati:
            if isinstance (diz_periodi[indice], str):
                blocco_testo_intervento = diz_periodi[indice]
                for counter, periodo_diaparser in enumerate([str(sentence).split('\n')[:-1] for sentence in parser.predict(blocco_testo_intervento, text='it').sentences]):
                    diz_periodi[indice + counter] = periodo_diaparser
        log['diz_periodi'] = diz_periodi ####
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        for indice in indici_periodi_analizzati:
            if isinstance (diz_periodi[indice][0], list):
                periodo_completa = []
                for num_blocco, blocco_periodo in enumerate(diz_periodi[indice]):
                    blocco_periodo[0] = blocco_periodo[0].replace('*',str(numero_periodo))
                    if num_blocco == 0:
                        numero_token_precedente = blocco_periodo[-1].split('\t')[0]
                        # se l'ultimo token del primo blocco è un mwt, basta risalire al numero del token precedente, 
                        # che sarà sicuramente un token semplice: non si danno mai due mwt di fila
                        if '-' in numero_token_precedente:
                            numero_token_precedente = int(blocco_periodo[-2].split('\t')[0])
                        else:
                            numero_token_precedente = int(numero_token_precedente)
                    else:
                        num_riga = 2
                        for riga in blocco_periodo[2:]:
                            numero_vecchio = riga.split('\t')[0]
                            if '-' in numero_vecchio:
                                primo_numero = numero_vecchio[:numero_vecchio.find('-')]
                                secondo_numero = numero_vecchio[numero_vecchio.find('-')+1:]
                                lunghezza_mwt = int(secondo_numero) - int(primo_numero)
                                nuovo_numero = str(numero_token_precedente+1) + '-' + str(numero_token_precedente+1+lunghezza_mwt)
                            else:
                                nuovo_numero = str(numero_token_precedente+1)
                                numero_token_precedente += 1
                            blocco_periodo[num_riga] = riga.replace(numero_vecchio, nuovo_numero, 1)
                            num_riga += 1
                    periodo_completa.extend(blocco_periodo)
                # Probabilmente devi tener conto del fatto che ci sono 
                # righe (quelle con l'asterisco) che non devono essere trattate come le altre e non sono necessariamente le prime due
                token_counter = annotazione_periodo(token_counter,periodo_completa)
            else:
                numero_periodo_annotato = re.search(r'[0-9]+', diz_periodi[indice][0])[0]
                diz_periodi[indice][0].replace(numero_periodo_annotato, str(numero_periodo))
                token_counter = annotazione_periodo(token_counter, diz_periodi[indice])
            numero_periodo += 1
    else:
        lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(testo_intervento, text='it').sentences]
        for periodo in lista_periodi:
            token_counter = annotazione_periodo(token_counter, periodo)
'''

In [ ]:
# backup prima del cambio della struttura dati delle periodi lunghe
r'''
def gestione_intervento(intervento):
    with open(percorso_file,'a', encoding='utf-8') as file:
        riga_iniziale = '# INTERVENTO NUMERO {}\n'.format(numero_intervento)
        file.write(riga_iniziale)
        file.write('# id="' + intervento.attrib['id'] + '"' + ' cognomeNome="' + intervento.attrib['cognomeNome'] + '"')
        if intervento.attrib['tipo'] == 'parlamentare':
            file.write(' partito="' + intervento.attrib['partito'] + '"\n')
        elif intervento.attrib['tipo'] in ('procedurale','extra_parlamentare'):
            file.write(' qualifica="' + intervento.attrib['qualifica'] + '"\n')
        else:
            file.write("\n")
    token_counter = 1
    diz_periodi = {} ##
    doc = nlp(intervento.text)
    indice_iniziale_periodo_precedente = 0
    for periodo in doc.sentences:
        if len(periodo.words) >= 490:
            numero_iterazioni = int(len(periodo.words)/490)
            if len(periodo.words)%490 != 0:
                numero_iterazioni += 1
            indice_parola = 0
            for i in range(numero_iterazioni):
                indice_carattere_iniziale = int(periodo.words[indice_parola].misc[periodo.words[indice_parola].misc.find("=")+1:periodo.words[indice_parola].misc.find("|")])
                if i != numero_iterazioni-1:
                    # indice_carattere_finale dev'essere escluso: è l'indice del primo carattere dopo la parola
                    indice_carattere_finale = int(periodo.words[indice_parola+489].misc[periodo.words[indice_parola+489].misc.rfind("=")+1:])
                    indice_parola += 490
                    testo = intervento.text[indice_carattere_iniziale:indice_carattere_finale]
                else:
                    testo = intervento.text[indice_carattere_iniziale:]
                    indice_carattere_finale = len(periodo.text) # va bene che prenda l'indice del primo carattere dopo questa periodo, perché indice_carattere_finale è così definito (vd. sopra)
                periodo_diaparser = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(testo, text='it').sentences]
                if len(periodo_diaparser) > 1:
                    print("Diaparser ha risplittato una parte di una sentence > 490 words (ottenuta con Stanza) :")
                    print("\t No. seduta: {}".format(seduta.attrib['numero']))
                    print("\t Id intervento: {}".format(intervento.attrib['id']))
                    print("Procedo a riunirle")
                    for periodo in periodo_diaparser[1:]:
                        periodo_diaparser[0].extend(periodo)
                    periodo_diaparser = periodo_diaparser[0]
                periodo_diaparser[0].insert(0,'# sent_id = 0.{}'.format(i+1)) # lo zero va poi corretto
                periodo_diaparser[0].insert(1, "# text = " + testo)
                if indice_carattere_iniziale != indice_iniziale_periodo_precedente:
                    diz_periodi[indice_iniziale_periodo_precedente] = intervento.text[indice_iniziale_periodo_precedente:indice_carattere_iniziale]
                diz_periodi[indice_carattere_iniziale] = periodo_diaparser
                indice_iniziale_periodo_precedente = indice_carattere_finale # forse ha senso tirare questi assegnamenti fuori perché all'interno del ciclo "for i in" non succederà mai che ci sia un'interruzione di una periodo non troppo lunga
    if bool(diz_periodi) is True:
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        counter_periodi = 1
        for indice in indici_periodi_analizzati:
            if isinstance (diz_periodi[indice], str):
                testo = diz_periodi[indice]
                for counter, periodo_diaparser in enumerate([str(sentence).split('\n')[:-1] for sentence in parser.predict(testo, text='it').sentences]):
                    dic_periodi[indice + counter] = periodo_diaparser
        indici_periodi_analizzati = sorted(diz_periodi.keys())
        for indice in indici_periodi_analizzati:
            if '0.' in diz_periodi[indice][0]:
                diz_periodi[indice][0] = # DA FINIRE, MA PENSO CAMBIERò IL CONTENUTO DEL DIZIONARIO NEL CASO DI periodi SPEZZATI: NON PIù X LISTE DI STRINGHE SEPARATE, MA UN'UNICA LISTA (?)
            token_counter = annotazione_periodo(token_counter, diz_periodi[indice])
        # vedi se il primo elemento della prima tupla è 0: 
        #    se sì, parti dal secondo elemento per spliceare il testo dell'intervento
        #    se no, parti da 0
        # splicea il testo dell'intervento in pezzi corrispondenti ai periodi più corti di 490 parole (secondo Stanza)
        # (ma forse lo splicing lo puoi fare direttamente nel blocco di codice precedente, dove trovi le periodi più lun-
        # ghe di 490 parole)
        # e fai analizzare a Diaparser ciascuno di questi pezzi
        # metti il risultato dell'analisi di diaparser nel dizionario lista_periodi
        # ordina il dizionario secondo l'ordine delle chiavi (ma forse l'ordine puoi darlo sempre nel blocco precedente)
        # prendi i valori del dizionario (cioè le liste che sono le periodi analizzate) e mettile in questo ciclo:
        #  for periodo in lista_periodi:
        #    token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
    else:
        lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(intervento.text, text='it').sentences]
        for periodo in lista_periodi:
            token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
            
            
    VECCHIA:

    if periodo_lungo:
        # MODIFICA: CON STANZA FAI SOLO LE periodi PIù LUNGHE DI 490 WORDS. BASTA SPEZZARE L'INTERVENTO IN NO. DI periodi
        # > DI 490 + PARTI RIMANENTI E ASSEGNARE LE PARTI RIMANENTI A COMANDI SEPARATI DI ANALISI. POI METTERE TUTTO INSIEME
        # IN UNA LISTA UNICA, RISPETTANDO PERò L'ORDINE NEL TESTO ORIGINARIO
        for counter, periodo in enumerate(doc.sentences):
            counter += 1
            if len(periodo.words) >= 490:
                numero_iterazioni = int(len(periodo.words)/490)
                if len(periodo.words)%490 != 0:
                    numero_iterazioni += 1
                indice_parola = 0
                for i in range(numero_iterazioni):
                    indice_carattere_iniziale = int(periodo.words[indice_parola].misc[periodo.words[indice_parola].misc.find("=")+1:periodo.words[indice_parola].misc.find("|")])
                    if i != numero_iterazioni-1:
                        # indice_carattere_finale dev'essere escluso: è l'indice del primo carattere dopo la parola
                        indice_carattere_finale = int(periodo.words[indice_parola+489].misc[periodo.words[indice_parola+489].misc.rfind("=")+1:])
                        indice_parola += 490
                        testo = intervento.text[indice_carattere_iniziale:indice_carattere_finale]
                    else:
                        testo = intervento.text[indice_carattere_iniziale:]
                    lista_periodi = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(testo, text='it').sentences]
                    if len(lista_periodi) > 1:
                        print("Diaparser ha risplittato una parte di una sentence > 490 words (ottenuta con Stanza) :")
                        print("\t No. seduta: {}".format(seduta.attrib['numero']))
                        print("\t Id intervento: {}".format(intervento.attrib['id']))
                        print("Procedo a riunirle")
                        for periodo in lista_periodi[1:]:
                            lista_periodi[0].extend(periodo)
                        lista_periodi = lista_periodi[0]
                    lista_periodi[0].insert(0,'# sent_id = {}.{} STANZA'.format(counter,i))
                    lista_periodi[0].insert(1, "# text = " + testo)
                    # è un po' strano continuare ad usare una lista quando sei sicuro che nella lista ci sia un
                    # solo elemento --> vedi se e come cambiare (ma è secondario)
                    for periodo in lista_periodi:
                        token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
                        # Da modificare: il numeratore dei token di Diaparser (che dev'essere trasversale per tutte le falsi periodi individuate così)
            else:
                lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(periodo.text, text='it').sentences]
                if len(lista_periodi) > 1:
                    print("Diaparser ha risplittato una sentence di Stanza (procedo a riunirle, ma andrà ricontrollata l'analisi sintattica):")
                    print("\t No. seduta: {}".format(seduta.attrib['numero']))
                    print("\t Id intervento: {}".format(intervento.attrib['id']))
                    print("\t Numero di periodi risultanti: {}".format(len(lista_periodi)))
                    print("\t Inizio e fine di ciascuna delle periodi risultanti:")
                    for cnt, periodo in enumerate(lista_periodi):
                        print("\t\t Inizio: '"+ periodo[1][periodo[1].find('=')+1:periodo[1].find('=')+11]+"...'")
                        print("\t\t Fine: '"+ periodo[1][-11:]+"...'")
                        if cnt > 0:
                            del periodo[0:2]
                            lista_periodi[0].extend(periodo)
                    lista_periodi = lista_periodi[0]
                del lista_periodi[0][0:2]
                lista_periodi[0].insert(0,'# sent_id = {}'.format(counter))
                lista_periodi[0].insert(1, "# text = " + periodo.text)
    else: 
'''          

In [5]:
# DA CANCELLARE, LO TENGO QUI SOLO PER BACKUP
'''
def gestione_intervento(intervento):
    with open(percorso_file,'a', encoding='utf-8') as file:
        riga_iniziale = '# INTERVENTO NUMERO {}\n'.format(numero_intervento)
        file.write(riga_iniziale)
        file.write('# id="' + intervento.attrib['id'] + '"' + ' cognomeNome="' + intervento.attrib['cognomeNome'] + '"')
        if intervento.attrib['tipo'] == 'parlamentare':
            file.write(' partito="' + intervento.attrib['partito'] + '"\n')
        elif intervento.attrib['tipo'] in ('procedurale','extra_parlamentare'):
            file.write(' qualifica="' + intervento.attrib['qualifica'] + '"\n')
        else:
            file.write("\n")
    token_counter = 1
    doc = nlp(intervento.text)
    periodo_lungo = False
    for counter, periodo in enumerate(doc.sentences):
        if len(periodo.words) >= 490:
            periodo_lungo = True
            break
    if periodo_lungo:
        # MODIFICA: CON STANZA FAI SOLO LE periodi PIù LUNGHE DI 490 WORDS. BASTA SPEZZARE L'INTERVENTO IN NO. DI periodi
        # > DI 490 + PARTI RIMANENTI E ASSEGNARE LE PARTI RIMANENTI A COMANDI SEPARATI DI ANALISI. POI METTERE TUTTO INSIEME
        # IN UNA LISTA UNICA, RISPETTANDO PERò L'ORDINE NEL TESTO ORIGINARIO
        for counter, periodo in enumerate(doc.sentences):
            counter += 1
            if len(periodo.words) >= 490:
                numero_iterazioni = int(len(periodo.words)/490)
                if len(periodo.words)%490 != 0:
                    numero_iterazioni += 1
                indice_parola = 0
                for i in range(numero_iterazioni):
                    indice_carattere_iniziale = int(periodo.words[indice_parola].misc[periodo.words[indice_parola].misc.find("=")+1:periodo.words[indice_parola].misc.find("|")])
                    if i != numero_iterazioni-1:
                        # indice_carattere_finale dev'essere escluso: è l'indice del primo carattere dopo la parola
                        indice_carattere_finale = int(periodo.words[indice_parola+489].misc[periodo.words[indice_parola+489].misc.rfind("=")+1:])
                        indice_parola += 490
                        testo = intervento.text[indice_carattere_iniziale:indice_carattere_finale]
                    else:
                        testo = intervento.text[indice_carattere_iniziale:]
                    lista_periodi = [str(sentence).split('\n')[2:-1] for sentence in parser.predict(testo, text='it').sentences]
                    if len(lista_periodi) > 1:
                        print("Diaparser ha risplittato una parte di una sentence > 490 words (ottenuta con Stanza) :")
                        print("\t No. seduta: {}".format(seduta.attrib['numero']))
                        print("\t Id intervento: {}".format(intervento.attrib['id']))
                        print("Procedo a riunirle")
                        for periodo in lista_periodi[1:]:
                            lista_periodi[0].extend(periodo)
                        lista_periodi = lista_periodi[0]
                    lista_periodi[0].insert(0,'# sent_id = {}.{} STANZA'.format(counter,i))
                    lista_periodi[0].insert(1, "# text = " + testo)
                    # è un po' strano continuare ad usare una lista quando sei sicuro che nella lista ci sia un
                    # solo elemento --> vedi se e come cambiare (ma è secondario)
                    for periodo in lista_periodi:
                        token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
                        # Da modificare: il numeratore dei token di Diaparser (che dev'essere trasversale per tutte le falsi periodi individuate così)
            else:
                lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(periodo.text, text='it').sentences]
                if len(lista_periodi) > 1:
                    print("Diaparser ha risplittato una sentence di Stanza (procedo a riunirle, ma andrà ricontrollata l'analisi sintattica):")
                    print("\t No. seduta: {}".format(seduta.attrib['numero']))
                    print("\t Id intervento: {}".format(intervento.attrib['id']))
                    print("\t Numero di periodi risultanti: {}".format(len(lista_periodi)))
                    print("\t Inizio e fine di ciascuna delle periodi risultanti:")
                    for cnt, periodo in enumerate(lista_periodi):
                        print("\t\t Inizio: '"+ periodo[1][periodo[1].find('=')+1:periodo[1].find('=')+11]+"...'")
                        print("\t\t Fine: '"+ periodo[1][-11:]+"...'")
                        if cnt > 0:
                            del periodo[0:2]
                            lista_periodi[0].extend(periodo)
                    lista_periodi = lista_periodi[0]
                del lista_periodi[0][0:2]
                lista_periodi[0].insert(0,'# sent_id = {}'.format(counter))
                lista_periodi[0].insert(1, "# text = " + periodo.text)
    else: 
        lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(intervento.text, text='it').sentences]
        for periodo in lista_periodi:
            token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
'''

<b>Prima dell'introduzione di Stanza</b>

In [2]:
numero_legislatura = str(18)
numero_seduta = str(6)
tree = ET.parse('Resoconti/legislatura_{}/seduta_{}.xml'.format(numero_legislatura, numero_seduta))
seduta = tree.getroot()
seduta_da_analizzare = False
if len(list(seduta.iter("intervento"))) > 0:
    seduta_da_analizzare = True

In [3]:
def scrittura_file(periodo, token_counter, indici_da_saltare, cue, scope):
    with open(percorso_file, 'a', encoding='utf-8') as file:
        file.write(periodo[0] + '\n' + periodo[1] + '\n')
        for riga in periodo[2:]:
            file.write(riga)
            if len(indici_da_saltare) > 0:
                numero_token_in_periodo = riga.split('\t')[0]
                da_annotare = True
                if '-' in numero_token_in_periodo:
                    file.write('\t_\t_\t_\n')
                    da_annotare = False
                else:
                    numero_token_in_periodo = int(numero_token_in_periodo)
                    for coppia_indici in indici_da_saltare:
                        if numero_token_in_periodo >= coppia_indici[0] and numero_token_in_periodo <= coppia_indici[1]:
                            file.write("\t{}\tDESCR.\tDESCR.\n".format(token_counter))
                            da_annotare = False
                            token_counter += 1
                        break
                if da_annotare:
                    file.write("\t{}\t{}\t{}\n".format(token_counter, cue, scope))
                    token_counter += 1
            else:
                if '-' in riga.split('\t')[0]:
                    file.write('\t_\t_\t_\n')
                else:
                    file.write("\t{}\t{}\t{}\n".format(token_counter, cue, scope))
                    token_counter += 1
    return token_counter

In [4]:
"""
'periodo' è una lista di stringhe. La lista è la periodo dell'intervento ottenuta con diaparser, in formato CONLL-X; 
le stringhe contenute sono invece le righe della periodo stessa (ottenute usando come separatore della periodo '\n')
"""
def annotazione_periodo(token_counter, periodo, lista_periodi):
    indici_da_saltare = []
    sequenza_descrittiva = {}
    for riga in periodo[2:]:
        sequenza_descrittiva[riga.split('\t')[0]] = riga.split('\t')[1]
    for match in re.finditer(r'(\(.*?\))', " ".join(sequenza_descrittiva.values())):
        print("È una sequenza descrittiva?[d/descr/no]")
        print(match[0])
        print()
        da_analizzare = input()
        input_errato = True
        while input_errato:
            if da_analizzare in ('d','D','descr.','descr','DESCR','DESCR.'):
                input_errato = False
                indice_chiave_iniziale = (" ".join(sequenza_descrittiva.values()))[:match.span()[0]+1].count(" ")
                indice_chiave_finale = (" ".join(sequenza_descrittiva.values()))[:match.span()[1]].count(" ")
                # print(list(sequenza_descrittiva.values())[indice_chiave_iniziale:indice_chiave_finale+1])##########
                # print(indice_chiave_iniziale)####################
                # print('-->' + list(sequenza_descrittiva.keys())[indice_chiave_iniziale])
                # print(indice_chiave_finale)####################
                # print('-->' + list(sequenza_descrittiva.keys())[indice_chiave_finale])
                indici_da_saltare.append((int(list(sequenza_descrittiva.keys())[indice_chiave_iniziale]), int(list(sequenza_descrittiva.keys())[indice_chiave_finale])))
            elif da_analizzare in ('N','n','no','No','NO','nO'):
                input_errato = False
            else:
                print("Input errato. Riprova [d/descr/no]:")
                print(match)
                da_analizzare = input()
    print(periodo[1][9:])
    da_analizzare = input("Vuoi analizzare questa periodo?[sì/no/forse]")
    input_errato = True
    while input_errato is True:
        if da_analizzare in ('N','n','no','No','NO','nO',''):
            input_errato = False
            token_counter = scrittura_file(periodo, token_counter, indici_da_saltare,'O','_')
            print("Hai risposto di no.")    
        elif da_analizzare in ('sì','si','Sì','s','S','SI'):
            input_errato = False
            token_counter = scrittura_file(periodo, token_counter, indici_da_saltare,' ',' ')
            print("Hai risposto di sì.")
        elif da_analizzare in ('dubbio', 'dub', 'forse','?'):
            input_errato = False
            token_counter = scrittura_file(periodo, token_counter, indici_da_saltare,'?','?')
            print("Hai risposto forse.")
        elif da_analizzare in ('descr.', 'DESCR.', 'd', 'descr', 'DESCR'):
            input_errato = False
            token_counter = scrittura_file(periodo, token_counter, indici_da_saltare, 'DESCR.','DESCR.')
            print("Hai risposto 'descrizione'.")
        else:
            print("Input errato. Riprova.")
            print(periodo[1][9:])
            da_analizzare = input("Vuoi analizzare questa periodo?[sì/no/forse]")
    return token_counter

In [5]:
def gestione_intervento(intervento):
    with open(percorso_file,'a', encoding='utf-8') as file:
        riga_iniziale = '# INTERVENTO NUMERO {}\n'.format(numero_intervento)
        file.write(riga_iniziale)
        file.write('# id="' + intervento.attrib['id'] + '"' + ' cognomeNome="' + intervento.attrib['cognomeNome'] + '"')
        if intervento.attrib['tipo'] == 'parlamentare':
            file.write(' partito="' + intervento.attrib['partito'] + '"\n')
        elif intervento.attrib['tipo'] in ('procedurale','extra_parlamentare'):
            file.write(' qualifica="' + intervento.attrib['qualifica'] + '"\n')
        else:
            file.write("\n")
    lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(intervento.text, text='it').sentences]
    token_counter = 1
    for periodo in lista_periodi:
        token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)

In [6]:
if seduta_da_analizzare:
    percorso_file = 'Resoconti_annotati/legislatura_{}/{}.tsv'.format(numero_legislatura, numero_seduta)
    file_esiste = False
    try:
        with open(percorso_file, 'x', encoding='utf-8') as file:
            legenda = "ID\tFORM\tLEMMA\tCPOS\tPOS\tFEATS\tHEAD\tDEPREL\tPHEAD\tPDEPREL\tID_TOKEN_NELL_INTERVENTO\tCUE\tSCOPE\n"
            file.write(legenda)
    except FileExistsError:
        file_esiste = True
        finisce_con_titolo = False
        with open(percorso_file, 'r', encoding='utf-8') as file:
            riga = file.readline()
            ultima_riga = ""
            while(riga != ""):
                if '# INTERVENTO NUMERO' in riga:
                    num_int_finale = int(re.search(r'[0-9]+', riga)[0])
                    num_periodo_finale = 0
                elif '# sent_id' in riga:
                    num_periodo_finale = int(re.search(r'[0-9]+', riga)[0])
                ultima_riga = riga
                riga = file.readline()
        if '# Titolo' in ultima_riga:
            finisce_con_titolo = True
        elif '# ' in ultima_riga:
            token_counter = 1
        else:
            token_counter = int(ultima_riga.split("\t")[10])+1
        if finisce_con_titolo is False:
            numero_intervento = num_int_finale
            intervento = list(seduta.iter("intervento"))[num_int_finale]
            lista_periodi = [str(sentence).split('\n')[:-1] for sentence in parser.predict(intervento.text, text='it').sentences][num_periodo_finale+1:]
            for periodo in lista_periodi:
                token_counter = annotazione_periodo(token_counter, periodo, lista_periodi)
    if file_esiste:
        numero_intervento = num_int_finale+1
        if finisce_con_titolo:
            el_iniziale = list(seduta.iter('intervento'))[num_int_finale+1]
            for counter, Element in enumerate(seduta.find('resoconto').iter()):
                if Element.tag == 'intervento' and Element.attrib['id'] == el_iniziale.attrib['id']:
                    num_el_iniziale = counter
                    break
        else:
            el_iniziale = list(seduta.iter('intervento'))[num_int_finale]
            for counter, Element in enumerate(seduta.find('resoconto').iter()):
                if Element.tag == 'intervento' and Element.attrib['id'] == el_iniziale.attrib['id']:
                    num_el_iniziale = counter + 1
                    break
        for Element in list(seduta.find('resoconto').iter())[num_el_iniziale:]:
            if Element.tag == 'intervento':
                gestione_intervento(Element)
                numero_intervento += 1
            elif Element.tag == 'dibattito':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file, 'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
            elif Element.tag == 'fase':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
    else:
        with open(percorso_file, 'a', encoding='utf-8') as file:
            file.write("#")
            for key, value in seduta.attrib.items():
                file.write(' ' + key + '="' + value +'"')
            file.write("\n")
        numero_intervento = 0
        for Element in list(seduta.find('resoconto').iter()):
            if Element.tag == 'intervento':
                gestione_intervento(Element)
                numero_intervento += 1
            elif Element.tag == 'dibattito':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")
            elif Element.tag == 'fase':
                span_titolo = str(len(list(Element.iter('intervento'))))
                with open(percorso_file,'a', encoding='utf-8') as file:
                    file.write("# Titolo per i prossimi {} interventi: ".format(span_titolo) + Element.find('titolo').text + "\n")

legge il processo verbale della seduta del 10 aprile 2018.
Vuoi analizzare questa periodo?[sì/no/forse]d
Hai risposto 'descrizione'.
Comunico che, ai sensi dell'articolo 46, comma 2, del Regolamento, i deputati Boschi, Cirielli, De Micheli, Ferri, Gregorio Fontana, Franceschini, Giacomelli, Lotti, Picchi e Pizzetti sono in missione a decorrere dalla seduta odierna.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
È una sequenza descrittiva?[d/descr/no]
( Ulteriori comunicazioni all' a l' Assemblea saranno pubblicate nell' in l' allegato A al a il resoconto della di la seduta odierna )

d
I deputati in missione sono complessivamente quattordici, come risulta dall'elenco depositato presso la Presidenza e che sarà pubblicato nell' allegato A al resoconto della seduta odierna (Ulteriori comunicazioni all'Assemblea saranno pubblicate nell'allegato A al resoconto della seduta odierna).
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
L'ordine del giorno reca l

Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Abbiamo, anzi, esplicitamente condizionato la nostra disponibilità ad attività di supporto logistico: le attività basate sui trattati bilaterali tra l'Italia e gli Stati Uniti del 1954 e del 1995.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Abbiamo condizionato questo supporto, che si è svolto in particolare dalla base aerea di Aviano, in provincia di Pordenone, al fatto che dal nostro territorio non partissero azioni dirette a colpire il territorio siriano, e così è stato.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Sul piano politico-diplomatico abbiamo chiarito ai nostri alleati, sin dall'inizio, la nostra contrarietà ad ogni escalation e abbiamo ribadito la convinzione italiana sulla impossibilità di giungere a una soluzione di un conflitto che si protrae da così lungo tempo soltanto con l'uso della forza e con l'idea di cacciare manu militari il dittatore Bashar al-Assad da

Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Ci sono in gioco i valori di libertà, di democrazia, i diritti, il libero commercio, e nessuna stagione sovranista può portare al tramonto dell'Occidente e dei suoi valori, almeno questa è la mia convinzione.
Vuoi analizzare questa periodo?[sì/no/forse]?
Hai risposto forse.
Scelta di campo, naturalmente, non vuol dire rinuncia all'autonomia e al perseguimento dei nostri interessi nazionali, non ha mai voluto dire questo.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Dobbiamo tranquillamente convincerci, perché questa è la storia del nostro Paese, e non da qualche mese, ma da molto tempo, che si può essere coerentemente nell'Alleanza Atlantica e, contemporaneamente, si possono marcare delle differenze, sottolineare degli interessi nazionali, perseguire degli obiettivi, magari mirando gradualmente a fare di questa impostazione l'impostazione che si rafforza e che coinvolge anche l'insieme dell'Alleanza.


Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
“L'Italia ripudia la guerra come strumento di offesa alla libertà degli altri popoli e come mezzo di risoluzione delle controversie internazionali”.
Vuoi analizzare questa periodo?[sì/no/forse]?
Hai risposto forse.
È una sequenza descrittiva?[d/descr/no]
( Applausi dei di i deputati del di il gruppo MoVimento 5 Stelle )

d
In questa cornice ci auguriamo anche che l'Unione europea sappia mostrarsi coesa e compatta ma che soprattutto l'ONU torni ad essere il tavolo al centro del quale portare le controversie internazionali e individuare le rispettive risoluzioni che, ripeto, devono essere diplomatiche e non militari, di pace e non di guerra (Applausi dei deputati del gruppo MoVimento 5 Stelle).
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
L'utilizzo di armi chimiche in Siria è semplicemente abominevole e a tal proposito ribadisco che la Convenzione sulla proibizione delle armi chimiche costituisce, per

Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
la Russia è parte della nostra storia, è inaccettabile che una parte dell'opinione pubblica, penso in particolare alla Gran Bretagna e agli Stati Uniti, torni a concetti di russofobia, che erano caratteristici dell'Ottocento e del grande gioco che avveniva in Asia tra l'Impero britannico e l'Impero russo.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
La storia è andata avanti.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
La Russia è parte di questa storia, il che non vuol dire che, se noi dovessimo essere chiamati a una scelta di campo, non saremmo fedeli alla nostra Alleanza atlantica, dove siamo e vogliamo essere attivi.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Ma in Siria non si è parlato di NATO, quindi sgombriamo il campo da questa questione NATO, che non c'è.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
È una sequenza descrittiva?[d

...

d
Pensiamoci, alla vigilia di scelte di governo che potrebbero decidere il futuro del mondo (Applausi dei deputati dei gruppi Forza Italia-Berlusconi Presidente e Fratelli d'Italia).
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Ha chiesto di parlare la deputata Giorgia Meloni.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Ne ha facoltà.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Grazie, Presidente.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Presidente Gentiloni, Fratelli d'Italia ha apprezzato la scelta del Governo italiano di non partecipare all'attacco in Siria, di non concedere le basi italiane per condurre quegli attacchi.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Ma, se abbiamo apprezzato quella scelta, se l'abbiamo condivisa, dobbiamo, invece, dirle che abbiamo parecchie riserve sulle parole che il Governo ha espresso anche in questa sede.
Vuoi analizzare questa periodo?[sì

Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Che nel centro di Barzah non vengono prodotte le armi chimiche, quindi qualcosa non torna, signori.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Che cos'è?
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Una cospirazione internazionale?
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Abbiamo, dopo le scie chimiche, le armi chimiche che vengono negate dagli operatori dell'Organizzazione contro le armi chimiche?
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Non so, per me qualcosa non torna.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Trump e Macron affermano che loro hanno le prove del fatto che quell'attacco c'è stato e che è stato Assad, però non le possono fare vedere, perché è questione di intelligence.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Theresa May, più prudente - come lei - parla di “evidenz

Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
È dalla nostra identità di nazione libera e democratica che dobbiamo trarre, nei momenti di difficoltà, il punto fondamentale per guardare avanti, dicendo con chiarezza comunque che per noi, per Liberi e Uguali - e anche questo deve essere posto con grande nettezza -, l'utilizzo delle nostre basi deve essere inibito ai mezzi che partecipano a raid e azioni militari in Siria.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
Come dicevo, è nei momenti di difficoltà che occorre andare a rileggere il testo base della nostra identità di nazione libera e democratica, che è la nostra Carta costituzionale.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
E per noi ogni azione, sia essa diplomatica, sia essa di peacekeeping, deve essere orientata dall'articolo 11 della nostra Costituzione, la Costituzione italiana, che quest'anno compie settant'anni.
Vuoi analizzare questa periodo?[sì/no/forse]
Ha

Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
la guerra in Siria non è iniziata con il bombardamento del 15 aprile, purtroppo la guerra in Siria è iniziata sette anni fa e ha già mietuto 350 mila vittime.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
È questa la domanda che tutto il mondo si deve porre: come contribuire alla fine di questa guerra, ma a creare una pace che sia duratura in tutto il mondo.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Questo è il ruolo che l'Italia deve avere, questo è il ruolo che l'Europa deve avere nell'adesione convinta al Patto Atlantico;
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
È una sequenza descrittiva?[d/descr/no]
( Applausi dei di i deputati del di il gruppo Misto - Noi con l' Italia e di deputati del di il gruppo Forza Italia - Berlusconi Presidente )

d
questo è il contributo che noi possiamo dare, senza falsi infingimenti e smettendola con le battaglie ideologi

Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
talché si presentano davanti all'ufficio postale o alla banca trepidanti, perché non sanno se gli daranno la stessa pensione o verrà decurtata.
Vuoi analizzare questa periodo?[sì/no/forse]s
Hai risposto di sì.
In questa situazione è particolarmente importante, e questo abbiamo rivendicato, che venga anche spiegato al pensionato cui viene diminuita la pensione, per quale motivo, in modo chiaro, semplice e umano direi, oltre che doveroso per l'Istituto nazionale della previdenza sociale.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
Debbo esprimere la mia riconoscenza e la mia gratitudine al presidente dell'INPS di Milano Via Silva, che si chiama Dotto e come dotto si è comportato, veramente, perché è un esempio per tanti altri funzionari pubblici.
Vuoi analizzare questa periodo?[sì/no/forse]
Hai risposto di no.
È una sequenza descrittiva?[d/descr/no]
( trascuro il nome e cognome per la privacy )


Alla 